In [3]:
from langchain.llms import OpenAI

# 环境配置

In [29]:
from dotenv import load_dotenv
import os

# 加载.env
load_dotenv('.env')

True

In [31]:
# 配置openai api key
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Building a Language Model Application: LLMs（构建语言模型应用程序：LLMs）
既然我们已经安装了LangChain并设置好了环境，那么我们就可以开始构建我们的语言模型应用程序。

LangChain提供了许多模块，可用于构建语言模型应用程序。这些模块可以组合在一起创建更复杂的应用程序，也可以单独使用于简单的应用。

## LLMs: Get predictions from a language model

In [5]:
llm = OpenAI(temperature=0.9)

In [6]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Rainbow Toes.


## Prompt Templates: Manage prompts for LLMs

In [10]:
from langchain.prompts import PromptTemplate

In [12]:
prompt = PromptTemplate(input_variables=['product'],template="What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [ ]:
prompt

## Chains: Combine LLMs and prompts in multi-step workflows(在多步骤工作流中结合LLMs和提示)

到目前为止，我们只是单独使用了PromptTemplate和LLM原语。但实际应用程序不仅仅是一个原语，而是它们的组合。

在LangChain中，链由链接组成，可以是像LLMs这样的基本元素或其他链。

最核心的链类型是LLMChain，它由PromptTemplate和LLM组成。

扩展之前的示例，我们可以构建一个LLMChain来接收用户输入、使用PromptTemplate格式化输入，并将格式化后的响应传递给LLM。

In [15]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [16]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.9)
prompt = PromptTemplate(
    input_variables=['product'],
    template="What is a good name for a company that makes {product}?"
)

In [17]:
chain = LLMChain(llm=llm,prompt=prompt)

In [18]:
chain.run("colorful socks")

'\n\nSocksational Colors.'

## Agents: Dynamically Call Chains Based on User Input(根据用户输入动态调用链)

到目前为止，我们看过的链条都是按照预定顺序运行的。

现在代理不再这样做：它们使用LLM来确定采取哪些操作以及以什么顺序进行。一个动作可以是使用工具并观察其输出，或者返回给用户。

如果正确使用代理，它们可以非常强大。在本教程中，我们将向您展示如何通过最简单、最高级别的API轻松地使用代理。

为了加载代理程序，您应该了解以下概念：

- 工具(Tool)：执行特定职责的函数。这可以是像Google搜索、数据库查找、Python REPL和其他链条之类的东西。工具的接口目前是期望有一个字符串作为输入，并且有一个字符串作为输出的函数。

- LLM：支持代理程序的语言模型。

- 代理程序(Agent)：要使用的代理程序。这应该是引用支持代理类别（class） 的字符串。因为本笔记本专注于最简单、最高级别API 的使用方式，所以只涵盖了标准支持代理程序（standard supported agents） 的使用方法 。如果您想实现自定义  代码，请参阅自定义代码文档 （即将推出） 。

Agents: 支持Agent及其规格列表，请参见此处

Tools: 预定义工具及其规格列表，请参见此处

In [9]:
# !pip install google-search-results

In [32]:
# 配置serp api key
os.environ['SERPAPI_API_KEY'] = os.getenv('SERPAPI_API_KEY')

In [8]:
from langchain.agents import load_tools,initialize_agent,AgentType
from langchain.llms import OpenAI

In [12]:
# 首先，让我们加载语言模型来控制代理。
llm = OpenAI(temperature=0)
# 接下来，让我们加载一些需要使用的工具。请注意，“llm-math” 工具使用了 LLM，因此我们需要传递该参数。
tools = load_tools(["serpapi","llm-math"],llm=llm)
# 最后，让我们使用工具、语言模型和所需的代理类型来初始化一个代理。verbose 设置是否显示详细信息
agent = initialize_agent(tools=tools,llm=llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

# 昨天旧金山的最高气温是多少华氏度？这个数字的0.23次方是多少？
# agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")
agent.run("昨天北京的最高气温是多少华氏度？这个数字的0.23次方是多少？")



> Entering new AgentExecutor chain...
 I need to find the temperature in Fahrenheit and then calculate the 0.23 power of that number.
Action: Search
Action Input: "Beijing temperature yesterday in Fahrenheit"
Observation: Beijing Temperature Yesterday. Maximum temperature yesterday: 65 °F (at 2:00 am) Minimum temperature yesterday: 57 °F (at 8:00 pm)
Thought: I need to calculate the 0.23 power of 65
Action: Calculator
Action Input: 65^0.23
Observation: Answer: 2.6119813521152198
Thought: I now know the final answer
Final Answer: 2.6119813521152198

> Finished chain.


'2.6119813521152198'

## Memory: Add State to Chains and Agents（为链和代理添加状态）

到目前为止，我们所使用的所有链和代理都是无状态的。但通常情况下，您可能希望某个链或代理具有一些“记忆”概念，以便它可以记住有关其先前交互的信息。最清晰和简单的例子就是设计聊天机器人 - 您希望它记住以前的消息，以便可以利用上下文进行更好的对话。这将是一种“短期记忆”。在更复杂的方面，您可以想象一个链/代理随着时间推移记住重要信息 - 这将是一种“长期记忆”的形式。有关后者更具体的想法，请参见这篇精彩论文。

LangChain提供了几个专门为此目的创建的链。本笔记本演示如何使用其中之一（ConversationChain）来实现两种不同类型的内存。

默认情况下，ConversationChain具有一种简单类型的内存，它会记录所有先前输入/输出并将其添加到传递给它们上下文中。让我们看看如何使用此链（设置verbose=True以便查看提示）。


In [13]:
from langchain import OpenAI,ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm,verbose=True)

conversation.predict(input="你好")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好
AI:

> Finished chain.


' 你好！很高兴见到你！我是一个智能AI，我可以回答你的问题，或者我们可以聊聊天？'

In [15]:
conversation.predict(input="我的名字叫：ZRRJDD")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好
AI:  你好！很高兴见到你！我是一个智能AI，我可以回答你的问题，或者我们可以聊聊天？
Human: 我的名字叫：ZRRJDD
AI:

> Finished chain.


' 嗨，ZRRJDD！很高兴认识你！你有什么想聊的？'

In [17]:
conversation.predict(input="你知道我的名字吗？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好
AI:  你好！很高兴见到你！我是一个智能AI，我可以回答你的问题，或者我们可以聊聊天？
Human: 我的名字叫：ZRRJDD
AI:  嗨，ZRRJDD！很高兴认识你！你有什么想聊的？
Human: 你知道我的名字吗？
AI:

> Finished chain.


' 不，我不知道你的名字。但是我很乐意认识你！'

# Building a Language Model Application: Chat Models（构建语言模型应用程序：聊天模型）
同样，您可以使用聊天模型代替LLMs。 聊天模型是语言模型的一种变体。虽然聊天模型在底层使用语言模型，但它们公开的接口略有不同：它们公开了一个“输入为聊天消息，输出也为聊天消息”的接口，而不是“文本输入、文本输出”的API。

由于聊天模型API相对较新，因此我们仍在摸索正确的抽象方法。

## Get Message Completions from a Chat Model（从聊天模型中获取消息完成度）
您可以通过向聊天模型传递一个或多个消息来获取聊天完成度。响应将是一条消息。目前在LangChain中支持的消息类型有AIMessage、HumanMessage、SystemMessage和ChatMessage——ChatMessage接受任意角色参数。大多数情况下，您只需要处理HumanMessage、AIMessage和SystemMessage。

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage,HumanMessage,SystemMessage

In [26]:
import json

In [20]:
chat = ChatOpenAI(temperature=0)

In [21]:
# 您可以通过传入单个消息来获得完成。
chat([HumanMessage(content="Translate this sentence from English to Chinese. I love programming.")])

AIMessage(content='我喜欢编程。', additional_kwargs={}, example=False)

In [23]:
# 您还可以为OpenAI的gpt-3.5-turbo和gpt-4模型传入多个消息。
messages =[
    SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
    HumanMessage(content="I love programming.")
]
chat(messages=messages)

AIMessage(content='我喜欢编程。', additional_kwargs={}, example=False)

In [25]:
# 您可以更进一步，使用generate为多组消息生成完成结果。这将返回一个带有额外消息参数的LLMResult：

batch_meesages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Chinese."),
        HumanMessage(content="I love artificial intelligence.")
    ]
]

result = chat.generate(batch_meesages)
for r in result:
    print(r)

('generations', [[ChatGeneration(text='我喜欢编程。', generation_info=None, message=AIMessage(content='我喜欢编程。', additional_kwargs={}, example=False))], [ChatGeneration(text='我喜欢人工智能。', generation_info=None, message=AIMessage(content='我喜欢人工智能。', additional_kwargs={}, example=False))]])
('llm_output', {'token_usage': {'prompt_tokens': 57, 'completion_tokens': 19, 'total_tokens': 76}, 'model_name': 'gpt-3.5-turbo'})
